In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
train_df.head()

,response_id,age,gender,years_at_company,job_role,monthly_income,work_life_balance,job_satisfaction,performance_rating,promotions_count,...,dependents_count,job_level,company_size,company_tenure,remote_work,leadership_opportunities,innovation_opportunities,company_reputation,employee_recognition,exit_status
0,8410,31,Male,19,Education,5390.0,Excellent,Medium,Average,2,...,0.0,Mid,Medium,89.0,No,No,No,Excellent,Medium,Stayed
1,64756,59,Female,4,Media,5534.0,Poor,High,Low,3,...,3.0,Mid,Medium,21.0,No,No,No,Fair,Low,Stayed
2,30257,24,Female,10,Healthcare,8159.0,Good,High,Low,0,...,3.0,Mid,Medium,74.0,No,No,No,Poor,Low,Stayed
3,65791,36,Female,7,Education,3989.0,Good,High,High,1,...,2.0,Mid,Small,50.0,Yes,No,No,Good,Medium,Stayed
4,65026,56,Male,41,Education,4821.0,NaN,NaN,Average,0,...,0.0,Senior,Medium,68.0,No,No,No,Fair,Medium,Stayed


In [4]:
train_df.drop_duplicates(inplace=True)

In [5]:
null_value_percentages=(train_df.isna().sum()/train_df.shape[0])*100
null_value_percentages

response_id                  0.000000
age                          0.000000
gender                       0.000000
years_at_company             0.000000
job_role                     0.000000
monthly_income               2.998423
work_life_balance           16.998893
job_satisfaction            11.998725
performance_rating           0.000000
promotions_count             0.000000
overtime                     0.000000
distance_from_home           0.000000
education_level              0.000000
marital_status               0.000000
dependents_count             7.998591
job_level                    0.000000
company_size                 4.998490
company_tenure               6.998557
remote_work                  0.000000
leadership_opportunities     0.000000
innovation_opportunities     0.000000
company_reputation           0.000000
employee_recognition         0.000000
exit_status                  0.000000
dtype: float64

In [6]:
columns_with_null_values = null_value_percentages[null_value_percentages>0]
print(columns_with_null_values)

monthly_income        2.998423
work_life_balance    16.998893
job_satisfaction     11.998725
dependents_count      7.998591
company_size          4.998490
company_tenure        6.998557
dtype: float64


In [7]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
label_encoder = LabelEncoder()
one_hot = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

In [8]:
# for columns in train_df.columns :
#     if train_df[columns].dtype==object:
#         train_df[columns] = label_encoder.fit_transform(train_df[columns])

In [9]:
train_df

,response_id,age,gender,years_at_company,job_role,monthly_income,work_life_balance,job_satisfaction,performance_rating,promotions_count,...,dependents_count,job_level,company_size,company_tenure,remote_work,leadership_opportunities,innovation_opportunities,company_reputation,employee_recognition,exit_status
0,8410,31,Male,19,Education,5390.0,Excellent,Medium,Average,2,...,0.0,Mid,Medium,89.0,No,No,No,Excellent,Medium,Stayed
1,64756,59,Female,4,Media,5534.0,Poor,High,Low,3,...,3.0,Mid,Medium,21.0,No,No,No,Fair,Low,Stayed
2,30257,24,Female,10,Healthcare,8159.0,Good,High,Low,0,...,3.0,Mid,Medium,74.0,No,No,No,Poor,Low,Stayed
3,65791,36,Female,7,Education,3989.0,Good,High,High,1,...,2.0,Mid,Small,50.0,Yes,No,No,Good,Medium,Stayed
4,65026,56,Male,41,Education,4821.0,NaN,NaN,Average,0,...,0.0,Senior,Medium,68.0,No,No,No,Fair,Medium,Stayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59593,37195,50,Female,12,Education,4414.0,Fair,High,Average,1,...,2.0,Senior,Small,35.0,No,No,Yes,Poor,Very High,Left
59594,6266,18,Male,4,Healthcare,8040.0,Fair,High,High,3,...,0.0,Senior,Medium,73.0,No,No,No,Fair,Medium,Left
59595,54887,22,Female,14,Technology,7944.0,Fair,High,High,0,...,2.0,Entry,Small,29.0,No,Yes,No,Good,Medium,Stayed
59596,861,23,Male,8,Education,2931.0,Fair,Very High,Average,0,...,0.0,Entry,Large,9.0,No,No,No,Good,Low,Left


In [10]:
train_df.drop(['response_id'],axis=1,inplace=True)

In [11]:
def calculate_outlier_percentage_for_column(df, column):
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in the DataFrame.")
    
    if not pd.api.types.is_numeric_dtype(df[column]):
        raise ValueError(f"Column '{column}' is not numeric.")

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()
    outlier_percentage = (outliers / len(df)) * 100
    return outlier_percentage

In [12]:
outliers=[]

In [13]:
for columns in train_df.columns:
    if train_df[columns].dtype!=object and calculate_outlier_percentage_for_column(train_df,columns):
         print(columns,calculate_outlier_percentage_for_column(train_df,columns))
         outliers.append(columns)

years_at_company 0.4580690627202256
monthly_income 0.09899661062451759
distance_from_home 0.008389543273264204


In [14]:
def remove_outlier(df,l):
    for column in l:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # df[column] = df[column].apply(lambda x: lower_bound if pd.notnull(x) and x < lower_bound else x)
        # df[column] = df[column].apply(lambda x: upper_bound if pd.notnull(x) and x > upper_bound else x)
        df[column] = df[column].apply(
            lambda x: min(max(x, lower_bound), upper_bound) if pd.notna(x) else x
        )
    return df

In [15]:
train_df = remove_outlier(train_df,outliers)

In [16]:
train_df.isna().sum()

age                             0
gender                          0
years_at_company                0
job_role                        0
monthly_income               1787
work_life_balance           10131
job_satisfaction             7151
performance_rating              0
promotions_count                0
overtime                        0
distance_from_home              0
education_level                 0
marital_status                  0
dependents_count             4767
job_level                       0
company_size                 2979
company_tenure               4171
remote_work                     0
leadership_opportunities        0
innovation_opportunities        0
company_reputation              0
employee_recognition            0
exit_status                     0
dtype: int64

In [17]:
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=5)
# train_df[['dependents_count','work_life_balance','job_satisfaction','company_size']]=imputer.fit_transform(train_df[['dependents_count','work_life_balance','job_satisfaction','company_size']])

In [18]:
train_df['monthly_income'].fillna(train_df['monthly_income'].mean(),inplace=True)
train_df['dependents_count'].fillna(train_df['dependents_count'].mode()[0],inplace=True)
train_df['company_tenure'].fillna(train_df['company_tenure'].mean(),inplace=True)
train_df['work_life_balance'].fillna(train_df['work_life_balance'].mode()[0],inplace=True)
train_df['job_satisfaction'].fillna(train_df['job_satisfaction'].mode()[0],inplace=True)
train_df['company_size'].fillna(train_df['company_size'].mode()[0],inplace=True)

/tmp/ipykernel_12144/1605568704.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['monthly_income'].fillna(train_df['monthly_income'].mean(),inplace=True)
/tmp/ipykernel_12144/1605568704.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col]

In [19]:
for column in train_df.columns:
    if train_df[column].dtype == object and column!='exit_status':  # Check for categorical columns
        # Reshape the column into a DataFrame
        reshaped_col = train_df[[column]]
        # Fit-transform and replace
        encoded = one_hot.fit_transform(reshaped_col)
        # Create a DataFrame for the encoded features
        feature_names = one_hot.get_feature_names_out([column])
        encoded_df = pd.DataFrame(encoded, columns=feature_names, index=train_df.index)
        # Drop the original column and concatenate the encoded features
        train_df = train_df.drop(column, axis=1).join(encoded_df)

In [20]:
train_df.head()

,age,years_at_company,monthly_income,promotions_count,distance_from_home,dependents_count,company_tenure,exit_status,gender_Female,gender_Male,...,innovation_opportunities_No,innovation_opportunities_Yes,company_reputation_Excellent,company_reputation_Fair,company_reputation_Good,company_reputation_Poor,employee_recognition_High,employee_recognition_Low,employee_recognition_Medium,employee_recognition_Very High
0,31,19.0,5390.0,2,22.0,0.0,89.0,Stayed,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,59,4.0,5534.0,3,21.0,3.0,21.0,Stayed,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,24,10.0,8159.0,0,11.0,3.0,74.0,Stayed,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,36,7.0,3989.0,1,27.0,2.0,50.0,Stayed,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,56,41.0,4821.0,0,71.0,0.0,68.0,Stayed,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [21]:
# new_outliers = []
# for columns in train_df.columns:
#     if train_df[columns].dtype!=object and calculate_outlier_percentage_for_column(train_df,columns):
#          print(columns,calculate_outlier_percentage_for_column(train_df,columns))
#          new_outliers.append(columns)

In [22]:
# train_df = remove_outlier(train_df,new_outliers)

In [23]:
train_df['exit_status']=train_df['exit_status'].map({'Stayed':1,'Left':0})

In [24]:
train_df.to_csv('part2_processed_train.csv',index=False)

In [25]:
test_df = pd.read_csv('test.csv')

In [26]:
test_df.head()

,response_id,age,gender,years_at_company,job_role,monthly_income,work_life_balance,job_satisfaction,performance_rating,promotions_count,...,marital_status,dependents_count,job_level,company_size,company_tenure,remote_work,leadership_opportunities,innovation_opportunities,company_reputation,employee_recognition
0,52685,36,Male,13,Healthcare,8029.0,Excellent,High,Average,1,...,Married,1.0,Mid,Large,22.0,No,No,No,Poor,Medium
1,30585,35,Male,7,Education,4563.0,Good,High,Average,1,...,Single,4.0,Entry,Medium,27.0,No,No,No,Good,High
2,54656,50,Male,7,Education,5583.0,Fair,High,Average,3,...,Divorced,2.0,Senior,Medium,76.0,No,No,Yes,Good,Low
3,33442,58,Male,44,Media,5525.0,NaN,NaN,High,0,...,Single,4.0,Entry,Medium,96.0,No,No,No,Poor,Low
4,15667,39,Male,24,Education,4604.0,Good,High,Average,0,...,Married,6.0,Mid,Large,45.0,Yes,No,No,Good,High


In [27]:
test_df.head()

,response_id,age,gender,years_at_company,job_role,monthly_income,work_life_balance,job_satisfaction,performance_rating,promotions_count,...,marital_status,dependents_count,job_level,company_size,company_tenure,remote_work,leadership_opportunities,innovation_opportunities,company_reputation,employee_recognition
0,52685,36,Male,13,Healthcare,8029.0,Excellent,High,Average,1,...,Married,1.0,Mid,Large,22.0,No,No,No,Poor,Medium
1,30585,35,Male,7,Education,4563.0,Good,High,Average,1,...,Single,4.0,Entry,Medium,27.0,No,No,No,Good,High
2,54656,50,Male,7,Education,5583.0,Fair,High,Average,3,...,Divorced,2.0,Senior,Medium,76.0,No,No,Yes,Good,Low
3,33442,58,Male,44,Media,5525.0,NaN,NaN,High,0,...,Single,4.0,Entry,Medium,96.0,No,No,No,Poor,Low
4,15667,39,Male,24,Education,4604.0,Good,High,Average,0,...,Married,6.0,Mid,Large,45.0,Yes,No,No,Good,High


In [28]:
test_df.isna().sum()

response_id                    0
age                            0
gender                         0
years_at_company               0
job_role                       0
monthly_income               447
work_life_balance           2533
job_satisfaction            1788
performance_rating             0
promotions_count               0
overtime                       0
distance_from_home             0
education_level                0
marital_status                 0
dependents_count            1192
job_level                      0
company_size                 745
company_tenure              1043
remote_work                    0
leadership_opportunities       0
innovation_opportunities       0
company_reputation             0
employee_recognition           0
dtype: int64

In [29]:
# test_df[['dependents_count','work_life_balance','job_satisfaction','company_size']]=imputer.fit_transform(test_df[['dependents_count','work_life_balance','job_satisfaction','company_size']])

In [30]:
test_df['dependents_count'].fillna(test_df['dependents_count'].mode()[0],inplace=True)
test_df['company_tenure'].fillna(test_df['company_tenure'].mean(),inplace=True)
test_df['monthly_income'].fillna(test_df['monthly_income'].mean(),inplace=True)
test_df['work_life_balance'].fillna(test_df['work_life_balance'].mode()[0],inplace=True)
test_df['job_satisfaction'].fillna(test_df['job_satisfaction'].mode()[0],inplace=True)
test_df['company_size'].fillna(test_df['company_size'].mode()[0],inplace=True)

/tmp/ipykernel_12144/2110389929.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['dependents_count'].fillna(test_df['dependents_count'].mode()[0],inplace=True)
/tmp/ipykernel_12144/2110389929.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

In [31]:
test_df.isna().sum()

response_id                 0
age                         0
gender                      0
years_at_company            0
job_role                    0
monthly_income              0
work_life_balance           0
job_satisfaction            0
performance_rating          0
promotions_count            0
overtime                    0
distance_from_home          0
education_level             0
marital_status              0
dependents_count            0
job_level                   0
company_size                0
company_tenure              0
remote_work                 0
leadership_opportunities    0
innovation_opportunities    0
company_reputation          0
employee_recognition        0
dtype: int64

In [32]:
for column in test_df.columns:
    if test_df[column].dtype == object:  # Check for categorical columns
        one_hot1 = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
        # Reshape the column into a DataFrame
        reshaped_col = test_df[[column]]
        # Fit-transform and replace
        encoded = one_hot1.fit_transform(reshaped_col)
        # Create a DataFrame for the encoded features
        feature_names = one_hot1.get_feature_names_out([column])
        encoded_df = pd.DataFrame(encoded, columns=feature_names, index=test_df.index)
        # Drop the original column and concatenate the encoded features
        test_df = test_df.drop(column, axis=1).join(encoded_df)

In [33]:
test_df.head()

,response_id,age,years_at_company,monthly_income,promotions_count,distance_from_home,dependents_count,company_tenure,gender_Female,gender_Male,...,innovation_opportunities_No,innovation_opportunities_Yes,company_reputation_Excellent,company_reputation_Fair,company_reputation_Good,company_reputation_Poor,employee_recognition_High,employee_recognition_Low,employee_recognition_Medium,employee_recognition_Very High
0,52685,36,13,8029.0,1,83,1.0,22.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,30585,35,7,4563.0,1,55,4.0,27.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,54656,50,7,5583.0,3,14,2.0,76.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,33442,58,44,5525.0,0,43,4.0,96.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,15667,39,24,4604.0,0,47,6.0,45.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [34]:
test_df.to_csv('part2_processed_test.csv',index=False)